In [67]:
# import modules
import pandas as pd
from datasets import Dataset, load_metric, ClassLabel
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

In [68]:
# set model constants and all that
batch_size=16
label_list=['O', 'B-CW', 'I-CW', 'B-PROD', 'I-PROD', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-CORP', 'I-CORP', 'B-GRP', 'I-GRP']

In [69]:
# also could create manual dict from labels but i have used `ClassLabel` 
CLASS_LABELS=ClassLabel(names=label_list,num_classes = len(label_list))

In [ ]:
# on indic-bert model page this is explained
tokenizer=AutoTokenizer.from_pretrained('ai4bharat/indic-bert')

In [71]:
# first task is to pre-process and create Dataset that can be stremed into the model
train_file_path='data/hi_train.conll'
dev_file_path='data/hi_dev.conll'

def get_dataset(file_path):
    """
    simple function to get ner data into nice dataset that can be used with indic-bert for tuning purposes
    """
    split_mark=' _ _ '
    tokens=[]
    ner_tags=[]
    with open(file_path,'r') as file:
        lines=file.readlines()
        tmp_tokens=[]
        tmp_ner_tags=[]
        for i,line in enumerate(lines):
            split_=line.split(split_mark)
            if len(split_)>1:
                tmp_tokens.append(split_[0])
                tmp_ner_tags.append(split_[1].strip())
            if line=='\n':
                if len(tmp_tokens)>0:
                    tokens.append(tmp_tokens)
                    ner_tags.append(tmp_ner_tags)
                tmp_tokens=[]
                tmp_ner_tags=[]
    
    df=pd.DataFrame({'tokens':tokens,'ner_tags':ner_tags})
    return Dataset.from_pandas(df)

train_dataset=get_dataset(train_file_path)
dev_dataset=get_dataset(dev_file_path)

In [ ]:
# need to tokenize these datasets to convert the tokens and labels into numeric form for our BERT model
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(CLASS_LABELS.str2int(label[word_idx]))
            else:
                label_ids.append(CLASS_LABELS.str2int(label[word_idx]) if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
dev_tokenized_datasets = dev_dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
# fine tune the model for ner task
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=len(label_list))

training_args = TrainingArguments(
    "test-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

In [74]:


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return results["overall_f1"]
    


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=dev_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:

trainer.evaluate()
trainer.save_model('hindi-ner.model')

In [78]:
import gc
gc.collect()

253